In [ ]:
!pip install pandas_ta

In [ ]:
!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install openpyxl

In [ ]:
!pip install ta

In [ ]:
# 📦 Install these if not already installed
# !pip install yfinance pandas_ta pandas tqdm gspread oauth2client

import yfinance as yf
import pandas as pd
import pandas_ta as ta
import random
import logging
from tqdm import tqdm
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ✅ Logging Setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ------------------------------ #
# 📥 Data Download
# ------------------------------ #
def download_data(ticker: str, start: str, end: str) -> pd.DataFrame:
    """
    Download historical stock data using yfinance.
    Args:
        ticker (str): Ticker symbol (e.g., "ASHOKLEY.NS")
        start (str): Start date (YYYY-MM-DD)
        end (str): End date (YYYY-MM-DD)
    Returns:
        pd.DataFrame: DataFrame with 'Close' and OHLCV data
    """
    df = yf.download(ticker, start=start, end=end)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]

    for col in df.columns:
        if 'close' in col.lower():
            df['Close'] = df[col]
            break
    else:
        raise KeyError("No suitable 'Close' column found.")

    df.dropna(inplace=True)
    logging.info(f"Downloaded data for {ticker}: {len(df)} rows")
    return df

# ------------------------------ #
# 📈 Add Technical Indicators
# ------------------------------ #
def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add technical indicators to the stock DataFrame.
    """
    df['RSI_14'] = ta.rsi(df['Close'], length=14)
    df['SMA_20'] = ta.sma(df['Close'], length=20)
    df['SMA_50'] = ta.sma(df['Close'], length=50)
    df['EMA_20'] = ta.ema(df['Close'], length=20)
    df['EMA_50'] = ta.ema(df['Close'], length=50)
    macd = ta.macd(df['Close'])
    df['MACD'] = macd['MACD_12_26_9']
    df.dropna(inplace=True)
    logging.info("Indicators added.")
    return df

# ------------------------------ #
# 🧠 Generate Strategy Rules
# ------------------------------ #
def generate_strategy() -> list:
    """
    Generate a random trading strategy rule pair.
    Returns:
        list: Two selected rules
    """
    rules = [
        ("RSI_14", "<", random.randint(10, 60)),
        ("RSI_14", ">", random.randint(40, 90)),
        ("MACD", ">", 0),
        ("MACD", "<", 0),
        ("SMA_20", ">", "SMA_50"),
        ("EMA_20", "<", "EMA_50"),
        ("Close", ">", "SMA_20"),
        ("Close", "<", "EMA_50")
    ]
    return random.sample(rules, 2)

# ------------------------------ #
# 📊 Apply Strategy Logic
# ------------------------------ #
def apply_strategy(df: pd.DataFrame, rule: list) -> pd.DataFrame:
    """
    Apply a strategy to the DataFrame using two conditions.
    Args:
        df (pd.DataFrame): Stock data
        rule (list): List of two conditions
    Returns:
        pd.DataFrame: Data with 'signal' column
    """
    df = df.copy()

    def evaluate(row, condition):
        indicator, operator, value = condition
        left = row[indicator]
        right = row[value] if isinstance(value, str) else value
        return (left > right) if operator == ">" else (left < right)

    cond1 = df.apply(lambda x: evaluate(x, rule[0]), axis=1)
    cond2 = df.apply(lambda x: evaluate(x, rule[1]), axis=1)
    df['signal'] = (cond1 & cond2).astype(int)
    return df

# ------------------------------ #
# 🧪 Backtest Strategy
# ------------------------------ #
def backtest(df: pd.DataFrame) -> tuple:
    """
    Backtest strategy based on generated signals.
    Returns:
        tuple: cumulative return and Sharpe ratio
    """
    df = df.copy()
    df['return'] = df['Close'].pct_change()
    df['strategy_return'] = df['return'] * df['signal'].shift(1)
    cumulative_return = (1 + df['strategy_return']).cumprod().iloc[-1]
    sharpe = 0.0
    if df['strategy_return'].std() != 0:
        sharpe = df['strategy_return'].mean() / df['strategy_return'].std() * (252 ** 0.5)
    return cumulative_return, sharpe

# ------------------------------ #
# 🏁 Run Strategy Engine
# ------------------------------ #
def run_engine(df: pd.DataFrame, n_strategies: int = 1000) -> pd.DataFrame:
    """
    Runs multiple strategies and stores performance.
    """
    results = []
    for _ in tqdm(range(n_strategies), desc="Running Strategies"):
        rule = generate_strategy()
        strat_df = apply_strategy(df, rule)
        cum_return, sharpe = backtest(strat_df)
        results.append({
            "rule_1": rule[0],
            "rule_2": rule[1],
            "cumulative_return": cum_return,
            "sharpe": sharpe
        })
    logging.info(f"Completed {n_strategies} strategy runs.")
    return pd.DataFrame(results)

# ------------------------------ #
# 📁 Save Top Strategies
# ------------------------------ #
def save_top_strategies(results: pd.DataFrame, top_n=20, filename="top_strategies.csv"):
    """
    Save top strategies to CSV file.
    """
    top = results.sort_values(by="sharpe", ascending=False).head(top_n)
    top.to_csv(filename, index=False)
    logging.info(f"Saved top {top_n} strategies to '{filename}'.")

# ------------------------------ #
# 📤 Google Sheets Logger
# ------------------------------ #
def log_to_google_sheets(trade_df: pd.DataFrame, result_df: pd.DataFrame,
                         sheet_name: str, creds_path: str):
    """
    Log signals and strategy results to Google Sheets.
    """
    trade_df = trade_df.copy()
    result_df = result_df.copy()

    for col in trade_df.columns:
        if pd.api.types.is_datetime64_any_dtype(trade_df[col]):
            trade_df[col] = trade_df[col].astype(str)
    if isinstance(trade_df.index, pd.DatetimeIndex):
        trade_df.index = trade_df.index.astype(str)

    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)

    try:
        sheet = client.open(sheet_name)
    except gspread.SpreadsheetNotFound:
        sheet = client.create(sheet_name)

    # Trade Log
    try:
        sheet.del_worksheet(sheet.worksheet("Trade_Log"))
    except:
        pass
    trade_ws = sheet.add_worksheet(title="Trade_Log", rows="1000", cols="20")
    trade_ws.update([trade_df.columns.tolist()] + trade_df.astype(str).values.tolist())

    # Strategy Summary
    try:
        sheet.del_worksheet(sheet.worksheet("Summary"))
    except:
        pass
    summary_ws = sheet.add_worksheet(title="Summary", rows="1000", cols="10")
    summary_ws.update([result_df.columns.tolist()] + result_df.astype(str).values.tolist())

    # Win Ratio
    win_df = result_df.copy()
    win_df['Win_Ratio'] = win_df['cumulative_return'].apply(lambda x: 1 if x > 1 else 0)
    summary = pd.DataFrame([{
        'Total Strategies': len(win_df),
        'Wins': win_df['Win_Ratio'].sum(),
        'Win Ratio': win_df['Win_Ratio'].mean()
    }])
    try:
        sheet.del_worksheet(sheet.worksheet("Win_Ratio"))
    except:
        pass
    win_ws = sheet.add_worksheet(title="Win_Ratio", rows="100", cols="10")
    win_ws.update([summary.columns.tolist()] + summary.astype(str).values.tolist())

    logging.info("✅ Logged data to Google Sheets.")

# ------------------------------ #
# 🤖 Main Algo Component
# ------------------------------ #
def run_algo(ticker="ASHOKLEY.NS", n_strategies=1000, start="2003-01-01", end="2025-01-01",
             sheet_name="QuantBacktest", creds_path='/content/secret-cipher-442319-s4-e4f2c438bb5f.json'):
    """
    Orchestrates the full pipeline.
    """
    logging.info(f"🚀 Starting Algo for {ticker}")
    df = download_data(ticker, start, end)
    df = add_indicators(df)
    results = run_engine(df, n_strategies)
    save_top_strategies(results)

    top_strategy = results.sort_values(by="sharpe", ascending=False).iloc[0]
    rule = [top_strategy["rule_1"], top_strategy["rule_2"]]
    signal_df = apply_strategy(df, rule)
    trade_log = signal_df[signal_df['signal'] == 1][['Close', 'signal']].reset_index()

    for col in trade_log.columns:
        if pd.api.types.is_datetime64_any_dtype(trade_log[col]):
            trade_log[col] = trade_log[col].astype(str)

    log_to_google_sheets(trade_log, results.reset_index(drop=True),
                         sheet_name=sheet_name, creds_path=creds_path)

# 🟩 Run Automatically
if __name__ == "__main__":
    run_algo()